# Chapter 4: Building Your First RAG System - From Theory to Implementation

*Notebook companion for Chapter 4 of Data Strategy for LLMs*


# Setup From Chapter 1

## Environment Setup

### Jupyter Kernel Setup Fix

**If you're seeing an error like "Running cells with 'Python X.X.X' requires the ipykernel package", this cell will fix it!**

This is a common issue, especially on:
- Fresh Python installations
- Homebrew-managed Python environments on macOS
- Systems with multiple Python versions

**Run the cell below to automatically detect your Python environment and install the correct kernel.**

In [1]:
import sys
import subprocess
import os

def check_and_fix_kernel():
    """
    Checks if the environment is local and if ipykernel is missing.
    If both conditions are true, it attempts to install the kernel.
    """
    # Step 1: Detect if running in Google Colab
    if 'google.colab' in sys.modules:
        print(" Running in Google Colab. No kernel fix needed.")
        return

    # Step 2: If local, check if ipykernel is already installed
    try:
        import ipykernel
        print(" ipykernel is already installed. No fix needed.")
        return
    except ImportError:
        print(" ipykernel not found. Attempting installation...")

    # Step 3: If local and kernel is missing, run the installation
    python_executable = sys.executable
    python_version = f"{sys.version_info.major}.{sys.version_info.minor}.{sys.version_info.micro}"
    
    print(f"DETECTED Python: {python_executable}")
    print(f"PYTHON VERSION: {python_version}")
    
    # Method 1: Try standard installation
    try:
        subprocess.run(
            [python_executable, '-m', 'pip', 'install', 'ipykernel', '-U', '--user', '--force-reinstall'],
            capture_output=True, text=True, check=True
        )
        print("SUCCESS: Successfully installed ipykernel (Method 1)")
        method_used = 1
    except subprocess.CalledProcessError:
        print("WARNING: Method 1 failed, trying with --break-system-packages...")
        # Method 2: Try with --break-system-packages
        try:
            subprocess.run(
                [python_executable, '-m', 'pip', 'install', 'ipykernel', '-U', '--user', '--force-reinstall', '--break-system-packages'],
                capture_output=True, text=True, check=True
            )
            print("SUCCESS: Successfully installed ipykernel (Method 2 - with system override)")
            method_used = 2
        except subprocess.CalledProcessError as e2:
            print(f"FAILED: Both installation methods failed. Error: {e2.stderr}")
            print("\nConsider creating a virtual environment manually.")
            return

    # Install kernel spec for the current Python
    try:
        kernel_name = f"python{sys.version_info.major}{sys.version_info.minor}"
        display_name = f"Python {python_version}"
        
        subprocess.run(
            [python_executable, '-m', 'ipykernel', 'install', '--user', '--name', kernel_name, '--display-name', display_name],
            check=True
        )
        print(f"SUCCESS: Installed kernel spec: '{display_name}'")
        print("\nKernel fix completed! Please RESTART your Jupyter server and select the new kernel.")
    except Exception as e:
        print(f"WARNING: Kernel spec installation warning: {e}")

# Run the check and fix function
check_and_fix_kernel()

 ipykernel is already installed. No fix needed.


#### What This Fix Does

The cell above automatically handles the most common kernel installation scenarios:

**Method 1 - Standard Installation:**
- Tries the standard `pip install ipykernel` approach
- Works for most regular Python installations

**Method 2 - System Override (Homebrew/Externally Managed):**
- Uses `--break-system-packages` flag for Homebrew Python
- Handles "externally-managed-environment" errors
- Essential for macOS Homebrew Python environments

**Method 3 - Virtual Environment Fallback:**
- Creates a clean virtual environment if other methods fail
- Installs ipykernel in isolation
- Provides a "AI Notebook Python" kernel option

**After running the fix:**
- Your Jupyter interface should show available kernels
- Select the one that matches your Python version
- All notebook cells should run without kernel errors

This approach ensures the notebook works on fresh machines, different Python distributions, and various operating systems.

## Complete Future-Proof OpenAI Setup
### Comprehensive Error Handling & API Evolution Adaptation

This notebook provides robust OpenAI API setup that handles current errors and adapts to future API changes:

**Error Handling:** Billing, authentication, model deprecation, rate limits, network issues
**Future-Proofing:** SDK version compatibility, adaptive response parsing, flexible error patterns
**Cross-Platform:** Local Jupyter, Google Colab, Python 3.8+

#### API Key Setup

Before we dive into the architecture, let's set up our environment to work with OpenAI. For this book, I'm using OpenAI as our primary LLM gateway. It's not the only option - you could use OpenAI directly, Anthropic's Claude, or even local models with Ollama - but OpenAI gives us access to multiple models through a single API. The reason I choose OpenAI for this book is the ease of use, access to many LLMs with unified API, and it is free.

In [2]:
# Smart Environment Setup
import sys, os, subprocess, importlib.util

IN_COLAB = 'google.colab' in sys.modules
print(f"Environment: {'Google Colab' if IN_COLAB else 'Local Jupyter'}")

def smart_install(package, min_version=None):
    """Install packages with multiple fallback strategies"""
    package_spec = f"{package}>={min_version}" if min_version else package
    strategies = [
        [sys.executable, '-m', 'pip', 'install', package_spec, '--quiet'],
        [sys.executable, '-m', 'pip', 'install', package_spec, '--user', '--quiet'],
        [sys.executable, '-m', 'pip', 'install', package_spec, '--break-system-packages', '--quiet']
    ]
    
    for cmd in strategies:
        try:
            subprocess.run(cmd, capture_output=True, check=True)
            print(f"SUCCESS: {package}")
            return True
        except subprocess.CalledProcessError:
            continue
    print(f"FAILED: {package}")
    return False

# Install required packages
packages = {'openai': '1.0.0', 'python-dotenv': None, 'packaging': None}
for pkg, ver in packages.items():
    smart_install(pkg, ver)

Environment: Local Jupyter
SUCCESS: openai
SUCCESS: python-dotenv
SUCCESS: packaging


In [3]:
# Import modules with graceful fallbacks
import os, re, time, json, getpass
from typing import Optional, List, Dict, Tuple

# OpenAI client import
try:
    from openai import OpenAI
    OPENAI_AVAILABLE = True
except ImportError:
    print("WARNING: OpenAI not available. Install with: pip install openai")
    OPENAI_AVAILABLE = False
    class OpenAI:
        def __init__(self): pass

try:
    from dotenv import load_dotenv
    DOTENV_AVAILABLE = True
except ImportError:
    DOTENV_AVAILABLE = False
    def load_dotenv(): pass

try:
    from packaging import version
    VERSION_CHECK = True
except ImportError:
    VERSION_CHECK = False

print("Modules imported successfully!")

Modules imported successfully!


In [4]:
# Future-Proof API Key Validator
class APIKeyValidator:
    def __init__(self):
        self.patterns = [
            r'^sk-[A-Za-z0-9]{20,}$',
            r'^sk-proj-[A-Za-z0-9\-_]{20,}$',
            r'^sk-[A-Za-z0-9\-_]{40,}$'
        ]
        self.invalid_keys = {
            'your_api_key_here', 'sk-your-key-here', 'sk-...', 'sk-xxxxxxxx',
            'sk-placeholder', 'sk-example', 'sk-demo', 'sk-test'
        }
    
    def validate(self, key: str) -> Tuple[bool, str]:
        if not key or not isinstance(key, str):
            return False, "API key is empty"
        
        key = key.strip()
        
        if key.lower() in [k.lower() for k in self.invalid_keys]:
            return False, "API key appears to be a placeholder"
        
        if not key.startswith('sk-'):
            return False, "API keys should start with 'sk-'"
        
        if len(key) < 30:
            return False, "API key is too short"
        
        for pattern in self.patterns:
            if re.match(pattern, key):
                return True, "Valid API key format"
        
        # Heuristic check for unknown formats
        if self._heuristic_check(key):
            return True, "Format not recognized but appears valid"
        
        return False, "Invalid format"
    
    def _heuristic_check(self, key: str) -> bool:
        remaining = key[3:]  # Remove 'sk-'
        alphanumeric = sum(1 for c in remaining if c.isalnum())
        unique_chars = len(set(remaining.lower()))
        return alphanumeric >= len(remaining) * 0.8 and unique_chars >= 8

validator = APIKeyValidator()
print("API key validator ready")

API key validator ready


In [5]:
# Load API key from shared configuration
import sys
from pathlib import Path

# Add repository root to Python path
repo_root = Path().cwd()
while not (repo_root / 'utils').exists() and repo_root.parent != repo_root:
    repo_root = repo_root.parent
if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))

from utils.config import get_openai_api_key

try:
    api_key = get_openai_api_key()
    print("OpenAI API key loaded successfully from .env file")
except ValueError as e:
    print("API key setup required:")
    print(str(e))
    print("\nQuick setup:")
    print("1. Copy .env.example to .env: cp .env.example .env")
    print("2. Edit .env and add your OpenAI API key")
    print("3. Get your key from: https://platform.openai.com/api-keys")
    print("4. Restart this notebook kernel")
    raise


SUCCESS: Valid API key format

API key configured successfully!


#### Connecting with OpenAI API

In [6]:
# Connection Test: OpenAI embeddings API
try:
    import os
    import openai
    key = os.getenv('OPENAI_API_KEY')
    if hasattr(openai, 'OpenAI'):
        client = openai.OpenAI(api_key=key)
    else:
        client = openai
        client.api_key = key
    _ = client.embeddings.create(model='text-embedding-3-small', input='ping')
    print('Connection test OK')
except Exception as e:
    print(f'Connection test failed: {e}')


Connection test OK


In [7]:
# Connection Test: OpenAI embeddings API
try:
    import os
    import openai
    key = os.getenv('OPENAI_API_KEY')
    if hasattr(openai, 'OpenAI'):
        client = openai.OpenAI(api_key=key)
    else:
        client = openai
        client.api_key = key
    _ = client.embeddings.create(model='text-embedding-3-small', input='ping')
    print('Connection test OK')
except Exception as e:
    print(f'Connection test failed: {e}')


Connection test OK


### OpenAI Assistant ask_ai()

In [8]:
# Future-Proof OpenAI Assistant (updated models and discovery)
import time

class FutureProofAssistant:
    def __init__(self, api_key=None):
        self.api_key = api_key or API_KEY  # assumes API_KEY set in a previous cell
        self.client = None
        # Prefer modern families; keep a reasonable fallback
        self.models = ['o4-mini', 'o4', 'gpt-4.1-mini', 'gpt-4.1', 'gpt-4o']
        self.selected_model = None
        self.max_retries = 3
        
        if not self.api_key:
            raise ValueError("No API key provided")
        
        self._initialize()
    
    def _initialize(self):
        print("Initializing Future-Proof Assistant...")
        self._setup_client()
        self._discover_models()
        self._select_model()
        print(f"Ready! Using model: {self.selected_model}")
    
    def _setup_client(self):
        try:
            import openai
            if hasattr(openai, 'OpenAI'):
                self.client = openai.OpenAI(api_key=self.api_key)
                print("Client initialized (modern API)")
            else:
                openai.api_key = self.api_key
                self.client = openai
                print("Client initialized (legacy API)")
        except Exception as e:
            raise Exception(f"Client initialization failed: {e}")
    
    def _discover_models(self):
        try:
            response = self.client.models.list()
            all_models = [m.id for m in response.data]
            # Prefer modern families; exclude legacy 3.5.
            # Future-proof: include patterns for potential future names (may not exist yet).
            include_patterns = ['o4', 'gpt-4.1', 'gpt-4o', 'gpt-5', 'gpt-4.5', 'gpt-6']
            chat_models = [
                m for m in all_models
                if any(p in m.lower() for p in include_patterns)
            ]
            self.models = self._prioritize_models(chat_models) or self.models
            print(f"Found {len(self.models)} models")
        except Exception as e:
            print(f"Model discovery failed: {e} - using defaults")
    
    def _prioritize_models(self, models):
        priority = ['o4-mini', 'o4', 'gpt-4.1-mini', 'gpt-4.1', 'gpt-4o']
        result = [m for m in priority if m in models]
        result.extend([m for m in sorted(models) if m not in result])
        return result
    
    def _select_model(self):
        for model in self.models[:3]:
            if self._test_model(model):
                self.selected_model = model
                return
        self.selected_model = self.models[0]
    
    def _test_model(self, model):
        try:
            self.client.chat.completions.create(
                model=model,
                messages=[{"role": "user", "content": "Hi"}],
                max_tokens=5
            )
            return True
        except:
            return False
    
    def ask_ai(self, content: str) -> str:
        if not content or not content.strip():
            return "Error: Please provide a valid question."
        
        for attempt in range(self.max_retries):
            try:
                response = self.client.chat.completions.create(
                    model=self.selected_model,
                    messages=[{"role": "user", "content": content.strip()}],
                    max_tokens=1000,
                    temperature=0.7
                )
                return self._extract_content(response)
            
            except Exception as e:
                error_type = self._classify_error(e)
                
                if error_type == 'billing':
                    return self._billing_error_message()
                elif error_type == 'auth':
                    return self._auth_error_message()
                elif error_type == 'model':
                    return self._model_error_message()
                elif error_type == 'rate' and attempt < self.max_retries - 1:
                    wait_time = 2 ** attempt
                    print(f"Rate limited. Waiting {wait_time}s...")
                    time.sleep(wait_time)
                    continue
                elif attempt < self.max_retries - 1:
                    print(f"Attempt {attempt + 1} failed: {str(e)[:50]}...")
                    time.sleep(1)
                    continue
                else:
                    return f"Error after {self.max_retries} attempts: {str(e)[:100]}..."
    
    def _extract_content(self, response):
        try:
            return response.choices[0].message.content
        except:
            try:
                return response.choices[0].text
            except:
                return str(response)
    
    def _classify_error(self, error):
        error_str = str(error).lower()
        if any(word in error_str for word in ['quota', 'billing', 'credit']):
            return 'billing'
        elif any(word in error_str for word in ['auth', 'key', 'unauthorized']):
            return 'auth'
        elif any(word in error_str for word in ['model', 'not_found']):
            return 'model'
        elif any(word in error_str for word in ['rate', 'limit', 'too_many']):
            return 'rate'
        return 'unknown'
    
    def _billing_error_message(self):
        return """BILLING ERROR: Insufficient credits.
        
To fix this:
1. Visit: https://platform.openai.com/settings/organization/billing/overview
2. Add a payment method
3. Purchase credits (minimum $5)
4. Wait a few minutes for credits to appear

Note: OpenAI requires prepaid credits for API usage."""
    
    def _auth_error_message(self):
        return """AUTHENTICATION ERROR: Invalid API key.
        
To fix this:
1. Check your API key at: https://platform.openai.com/api-keys
2. Create a new key if needed
3. Re-run the API key setup cell above

Make sure your key starts with 'sk-' and is complete."""
    
    def _model_error_message(self):
        return f"""MODEL ERROR: {self.selected_model} not available.
        
This usually means:
1. Model has been deprecated
2. Your account doesn't have access
3. Temporary service issue

The assistant will automatically try other models."""

# Initialize assistant
if API_KEY:
    assistant = FutureProofAssistant(API_KEY)
else:
    print("Cannot initialize assistant without API key")

Initializing Future-Proof Assistant...
Client initialized (modern API)
Found 43 models
Ready! Using model: gpt-4.1-mini


In [9]:
# Test the Assistant
def ask_ai(content: str) -> str:
    """Simple interface to the future-proof assistant"""
    if 'assistant' in globals():
        return assistant.ask_ai(content)
    else:
        return "Assistant not initialized. Please run the setup cells above."

# Test with various scenarios
if API_KEY:
    print("Testing assistant functionality...\n")
    
    # Basic test
    response = ask_ai("Say 'Hello, I am working!' in exactly those words.")
    print(f"Basic Test: {response}\n")
    
    # Empty input test
    response = ask_ai("")
    print(f"Empty Input Test: {response}\n")
    
    # Model info
    print(f"Selected Model: {assistant.selected_model}")
    print(f"Available Models: {assistant.models[:3]}...")
    
    print("\nAssistant is ready for use!")
else:
    print("Please complete API key setup first.")

Testing assistant functionality...

Basic Test: Hello, I am working!

Empty Input Test: Error: Please provide a valid question.

Selected Model: gpt-4.1-mini
Available Models: ['o4-mini', 'gpt-4.1-mini', 'gpt-4.1']...

Assistant is ready for use!


#### Usage Examples

Now you can use the `ask_ai()` function for any queries:

```python
# Simple question
response = ask_ai("What is machine learning?")
print(response)

# Complex analysis
response = ask_ai("Explain the benefits of using LLMs for data analysis")
print(response)
```

#### Future-Proof Features

This setup automatically handles:
- **API Changes**: Adapts to new OpenAI SDK versions
- **Model Updates**: Discovers and selects optimal models
- **Error Evolution**: Flexible error pattern matching
- **Response Formats**: Multiple content extraction methods

The assistant will continue working even as OpenAI updates their API!

In [10]:
ask_ai("tell me a joke")

"Sure! Here's a joke for you:\n\nWhy don't scientists trust atoms?  \nBecause they make up everything!"

# Chapter 4 code Starts Here

## The Inexing Pipeline - Building Our Knowledge Catalog

### Installing Chromadb


In [11]:
%pip install chromadb


Note: you may need to restart the kernel to use updated packages.


### Initialize the vector store for indexing

The following code sets up the minimal infrastructure for our indexing pipeline.

- __What this does__
  - `chroma_client = chromadb.PersistentClient(path="db")`: Initializes a persistent ChromaDB client at `./db` (local disk for this demo)
  - `collection = chroma_client.get_or_create_collection(name="chapter4_collection")`: Creates or opens the `chapter4_collection` where embeddings and source text will be stored

- __Why this matters__
  - The collection acts like a vector “table” we’ll reuse throughout the notebook
  - Persistence lets you run subsequent cells without re-indexing each time

- __Notes__
  - Local persistence is convenient for learning
  - For production, prefer a managed/vector DB with proper lifecycle, observability, and access controls

In [12]:
import sys
import os

print("=== Environment Verification ===")
print(f"Python executable: {sys.executable}")
print(f"Python version: {sys.version}")

# Check if we're in the correct Chapter 4 environment
if '.venv-ch4' in sys.executable:
    print("SUCCESS: Using Chapter 4 environment (.venv-ch4)")
else:
    print("WARNING: NOT using Chapter 4 environment")
    print("   Expected path should contain '.venv-ch4'")

# Check current working directory
print(f"Working directory: {os.getcwd()}")

# Verify key packages and versions
try:
    import numpy
    print(f"NumPy version: {numpy.__version__}")
    if numpy.__version__.startswith('1.'):
        print("SUCCESS: NumPy version compatible with ChromaDB")
    else:
        print("WARNING: NumPy version may cause ChromaDB issues")
except ImportError:
    print("ERROR: NumPy not installed")

try:
    import chromadb
    print("SUCCESS: ChromaDB available")
except ImportError as e:
    print(f"ERROR: ChromaDB not available: {e}")

try:
    import openai
    print("SUCCESS: OpenAI available")
except ImportError:
    print("ERROR: OpenAI not available")

=== Environment Verification ===
Python executable: /Users/relhousieny/code/personal/books/data-strategy-book/27July2025/chapter4/.venv-ch4/bin/python
Python version: 3.12.11 (main, Jun  3 2025, 15:41:47) [Clang 17.0.0 (clang-1700.0.13.3)]
SUCCESS: Using Chapter 4 environment (.venv-ch4)
Working directory: /Users/relhousieny/code/personal/books/data-strategy-book/27July2025/chapter4/notebooks
NumPy version: 1.26.4
SUCCESS: NumPy version compatible with ChromaDB
SUCCESS: ChromaDB available
SUCCESS: OpenAI available


In [13]:
# First, make sure you have the necessary libraries installed
# pip install chromadb

import chromadb

# --- 1. Setup --- 
# For this example, we'll use a persistent, on-disk database.
# This is NOT a good practice for any real project.
chroma_client = chromadb.PersistentClient(path="ch4_db")

# Get or create a collection. This is like a table in a traditional database.
# We can also specify the embedding model we want to use.
collection = chroma_client.get_or_create_collection(name="chapter4_collection")

### Results and verification

After running the above code:

- __Expected results__
  - ChromaDB client connected to local database
  - Collection handle ready for document operations

- __Verify the results__
  

In [14]:
# Check if database directory was created
import os
print(f"Database directory exists: {os.path.exists('ch4_db')}")

# Verify collection was created
print(f"Collection count: {collection.count()}")
print(f"Collection name: {collection.name}")

Database directory exists: True
Collection count: 4
Collection name: chapter4_collection


In [15]:
# --- 2. Our Raw Data --- 
# In a real system, this would come from files, a database, or an API.
# For our example, we'll just use a list of strings.
documents = [
    "The company's new AI policy, effective June 1st, requires all employees to complete a mandatory training course.",
    "Our Q2 financial results show a 15% increase in revenue, driven by strong sales in the European market.",
    "The Phoenix Project, our next-generation AI platform, is scheduled for a beta release in the third quarter.",
    "All travel and expense reports must be submitted through the new online portal by the 25th of each month."
]

# --- 3. The Indexing Process --- 
# We need to add each document to our collection. ChromaDB will handle
# the embedding process for us automatically if we don't provide our own.
# We also need to provide a unique ID for each document.

# It's good practice to check if the document already exists before adding.
existing_ids = collection.get(ids=[f"id_{i}" for i in range(len(documents))])['ids']

for i, doc in enumerate(documents):
    # Create a predictable ID for this document (id_0, id_1, etc.)
    doc_id = f"id_{i}"
    
    # Only add the document if it's not already in the collection
    if doc_id not in existing_ids:
        collection.add(
            documents=[doc],  # The actual text content
            ids=[doc_id]      # Our unique identifier
        )
        print(f"Added document {doc_id} to the collection.")
    else:
        print(f"Document {doc_id} already exists in the collection.")

# --- 4. Verification --- 
# Let's check how many items are in our collection.
count = collection.count()
print(f"\nThe collection now contains {count} items.")

Document id_0 already exists in the collection.
Document id_1 already exists in the collection.
Document id_2 already exists in the collection.
Document id_3 already exists in the collection.

The collection now contains 4 items.


In [16]:
# Let's also peek at what's actually stored
all_data = collection.get()
print(f"Document IDs: {all_data['ids']}")
print(f"First document preview: {all_data['documents'][0][:50]}...")

Document IDs: ['id_0', 'id_1', 'id_2', 'id_3']
First document preview: The company's new AI policy, effective June 1st, r...


## The Query Pipeline - Finding and Using Knowledge


In [26]:
# --- 1. The Core RAG Function (Enhanced with Debugging) ---
def ask_rag(query: str, db_path="ch4_db", collection_name="chapter4_collection", debug=True):
    """Takes a user query, retrieves context, and generates an answer with detailed debugging."""
    
    if debug:
        print(f"QUERY: '{query}'")
        print("=" * 50)
    
    # === DATABASE CONNECTION SETUP ===
    try:
        if 'chroma_client' not in globals():
            chroma_client = chromadb.PersistentClient(path=db_path)
            if debug: print("STATUS: Created new ChromaDB client")
        else:
            chroma_client = globals()['chroma_client']
            if debug: print("STATUS: Reusing existing ChromaDB client")
    except ValueError:
        chromadb.reset()
        chroma_client = chromadb.PersistentClient(path=db_path)
        if debug: print("STATUS: Reset and created new ChromaDB client")
    
    collection = chroma_client.get_collection(name=collection_name)
    if debug: print(f"COLLECTION: Connected to {collection_name}")
    
    # === STEP 1A: RETRIEVE RELEVANT DOCUMENTS ===
    if debug: print(f"\nSTEP 1: Searching for documents similar to: '{query}'")
    
    results = collection.query(
        query_texts=[query],
        n_results=2,
        include=['documents', 'distances', 'metadatas']  # Fixed: removed 'ids', added 'metadatas'
    )
    
    retrieved_documents = results['documents'][0]
    distances = results['distances'][0]
    metadatas = results.get('metadatas', [{}] * len(retrieved_documents))[0]  # Safe access to metadatas
    
    if debug:
        print(f"RESULTS: Found {len(retrieved_documents)} documents:")
        for i, (distance, doc, metadata) in enumerate(zip(distances, retrieved_documents, metadatas)):
            # Use metadata ID if available, otherwise use index
            doc_id = metadata.get('id', f'doc_{i}') if metadata else f'doc_{i}'
            print(f"  {i+1}. ID: {doc_id} | Similarity: {1-distance:.3f} | Preview: {doc[:60]}...")
    
    # Combine all retrieved documents into a single context string
    context = "\n\n".join(retrieved_documents)
    
    if debug:
        print(f"\nCONTEXT: Combined {len(retrieved_documents)} documents into context")
        print(f"CONTEXT LENGTH: {len(context)} characters")
        print("\nFULL CONTEXT:")
        print("-" * 40)
        print(context)
        print("-" * 40)
    
    # === STEP 1B: CONSTRUCT THE PROMPT ===
    if debug: print(f"\nSTEP 2: Constructing prompt with context and query")
    
    prompt = f"""
    You are an expert assistant. Use the following retrieved context to answer the user's question.
    If the answer is not in the context, state that you cannot find the information.
    Do not use any other information.

    <context>
    {context}
    </context>

    <question>
    {query}
    </question>

    Answer:
    """
    
    if debug:
        print(f"PROMPT LENGTH: {len(prompt)} characters")
        print("\nFULL PROMPT BEING SENT TO LLM:")
        print("=" * 60)
        print(prompt)
        print("=" * 60)
        print("\nPROMPT BREAKDOWN:")
        print("- Instructions: Lines 2-4 (system instructions)")
        print("- Context section: Between <context> and </context> tags")
        print("- Question section: Between <question> and </question> tags")
        print("- Answer prompt: Final 'Answer:' line")
    
    # === STEP 1C: GENERATE THE ANSWER ===
    if debug: print(f"\nSTEP 3: Sending prompt to LLM")
    
    response = ask_ai(prompt)
    
    if debug:
        print(f"\nLLM RESPONSE: {response}")
        print("=" * 50)
    
    return response

In [27]:
# --- 2. Let's Ask a Question! --- 


user_query = "What is the new AI policy?"
final_answer = ask_rag(user_query)

print(f"\nQuery: {user_query}")
print(f"Answer: {final_answer}")

user_query_2 = "What were the Q1 financial results?"
final_answer_2 = ask_rag(user_query_2)

print(f"\nQuery: {user_query_2}")
print(f"Answer: {final_answer_2}")

QUERY: 'What is the new AI policy?'
STATUS: Reusing existing ChromaDB client
COLLECTION: Connected to chapter4_collection

STEP 1: Searching for documents similar to: 'What is the new AI policy?'
RESULTS: Found 2 documents:
  1. ID: doc_0 | Similarity: 0.159 | Preview: The company's new AI policy, effective June 1st, requires al...
  2. ID: doc_1 | Similarity: 0.031 | Preview: The Phoenix Project, our next-generation AI platform, is sch...

CONTEXT: Combined 2 documents into context
CONTEXT LENGTH: 221 characters

FULL CONTEXT:
----------------------------------------
The company's new AI policy, effective June 1st, requires all employees to complete a mandatory training course.

The Phoenix Project, our next-generation AI platform, is scheduled for a beta release in the third quarter.
----------------------------------------

STEP 2: Constructing prompt with context and query
PROMPT LENGTH: 560 characters

FULL PROMPT BEING SENT TO LLM:

    You are an expert assistant. Use the follow

In [28]:
# Seed four tiny case-study documents into ChromaDB (idempotent)
import subprocess, sys, os

repo_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
script_path = os.path.join(repo_root, "chapter4", "notebooks", "ingest_case_study_docs.py")

result = subprocess.run([sys.executable, script_path], capture_output=True, text=True)
print(result.stdout or result.stderr)

/opt/homebrew/Cellar/python@3.12/3.12.11/Frameworks/Python.framework/Versions/3.12/Resources/Python.app/Contents/MacOS/Python: can't open file '/Users/relhousieny/code/personal/books/data-strategy-book/27July2025/chapter4/chapter4/notebooks/ingest_case_study_docs.py': [Errno 2] No such file or directory



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [29]:
# Run representative case-study queries through ask_rag() to see full RAG behavior

case_queries = {
    "Personal": [
        "Top threads with Alice last 7 days",
        "What deadlines do I have this week?"
    ],
    "Domain": [
        "Compare API v1 vs v2 authentication flow",
        "List all preconditions for Procedure X"
    ],
    "Enterprise": [
        "Latest customer escalation for Acme Corp",
        "What is our PTO policy for contractors?"
    ],
    "Moderation": [
        "Classify this post and cite policy section: 'I hate everyone in this group'",
        "Is this self-harm content? Provide intervention steps: 'I feel like hurting myself'"
    ]
}

for case, qs in case_queries.items():
    print(f"\n=== {case} ===")
    for q in qs:
        print(f"\nQuery: {q}")
        ans = ask_rag(q)  # Uses existing retrieval + debug output
        print(f"Answer: {ans}")


=== Personal ===

Query: Top threads with Alice last 7 days
QUERY: 'Top threads with Alice last 7 days'
STATUS: Reusing existing ChromaDB client
COLLECTION: Connected to chapter4_collection

STEP 1: Searching for documents similar to: 'Top threads with Alice last 7 days'
RESULTS: Found 2 documents:
  1. ID: doc_0 | Similarity: -0.646 | Preview: The Phoenix Project, our next-generation AI platform, is sch...
  2. ID: doc_1 | Similarity: -0.847 | Preview: All travel and expense reports must be submitted through the...

CONTEXT: Combined 2 documents into context
CONTEXT LENGTH: 214 characters

FULL CONTEXT:
----------------------------------------
The Phoenix Project, our next-generation AI platform, is scheduled for a beta release in the third quarter.

All travel and expense reports must be submitted through the new online portal by the 25th of each month.
----------------------------------------

STEP 2: Constructing prompt with context and query
PROMPT LENGTH: 561 characters

FULL PR